In [ ]:
import os, numpy as np, pandas as pd, tensorflow as tf
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)
sns.set_context("notebook")

BASE_DIR = "/content/drive/MyDrive/UCI_HAR_Dataset/"

def load_file(path):
    return pd.read_csv(path, header=None, sep=r"\s+").values

def load_group(group, base):
    p = os.path.join(base, group, "Inertial Signals")
    names = [f"total_acc_x_{group}.txt", f"total_acc_y_{group}.txt", f"total_acc_z_{group}.txt",
             f"body_acc_x_{group}.txt",  f"body_acc_y_{group}.txt",  f"body_acc_z_{group}.txt",
             f"body_gyro_x_{group}.txt", f"body_gyro_y_{group}.txt", f"body_gyro_z_{group}.txt"]
    arrays = [load_file(os.path.join(p, n)) for n in names]
    return np.dstack(arrays)

In [ ]:
def load_dataset(base=BASE_DIR):
    Xtr = load_group("train", base); ytr = load_file(os.path.join(base, "train", "y_train.txt"))
    Xte = load_group("test",  base); yte = load_file(os.path.join(base, "test",  "y_test.txt"))
    ytr, yte = ytr - 1, yte - 1
    ytr_oh, yte_oh = to_categorical(ytr), to_categorical(yte)
    print(Xtr.shape, ytr.shape, ytr_oh.shape, Xte.shape, yte.shape, yte_oh.shape)
    return Xtr, ytr, ytr_oh, Xte, yte, yte_oh

trainX, trainy_int, trainy, testX, testy_int, testy = load_dataset(BASE_DIR)
X_train, X_val, y_train, y_val = train_test_split(
    trainX, trainy, test_size=0.2, random_state=SEED, stratify=trainy_int
)
nT, nC, nY = X_train.shape[1], X_train.shape[2], y_train.shape[1]
print(f"Input: (T={nT}, C={nC}), Classes={nY}")

In [ ]:
def build_cnn_lstm(nT, nC, nY):
    inp = Input((nT, nC))
    x = Conv1D(64, 5, padding='same', activation='relu')(inp)
    x = MaxPooling1D(2)(x)                      # (N, 64, 64)
    x = Conv1D(128, 3, padding='same', activation='relu')(x)
    x = LSTM(64)(x)                             # Conv 출력(3D)을 LSTM에 바로
    x = Dropout(0.2)(x)
    out = Dense(nY, activation='softmax')(x)
    return Model(inp, out, name="cnn_lstm")

model = build_cnn_lstm(nT, nC, nY)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cbs = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(patience=3, factor=0.5, verbose=1, monitor='val_loss')
]
hist = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                 epochs=30, batch_size=32, callbacks=cbs, verbose=1)

In [ ]:
loss, val_loss = hist.history['loss'], hist.history['val_loss']
acc,  val_acc  = hist.history['accuracy'], hist.history['val_accuracy']
ep = range(1, len(loss)+1)
plt.figure(figsize=(10,3.8))
plt.subplot(1,2,1); plt.plot(ep, loss, label='train'); plt.plot(ep, val_loss, label='val'); plt.title('Loss - CNN+LSTM'); plt.legend()
plt.subplot(1,2,2); plt.plot(ep, acc,  label='train'); plt.plot(ep, val_acc,  label='val'); plt.title('Accuracy - CNN+LSTM'); plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
prob = model.predict(testX, verbose=0)
pred = prob.argmax(axis=-1)
acc_test = accuracy_score(testy_int, pred)
print(f"\n== CNN+LSTM Test Accuracy: {acc_test:.4f}")
print(classification_report(testy_int, pred, digits=4))
cm = confusion_matrix(testy_int, pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - CNN+LSTM'); plt.xlabel('Predicted'); plt.ylabel('True')
plt.tight_layout(); plt.show()